In [1]:
import numpy as np
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage
import skimage
from skimage import io
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_ubyte

import cv2
from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma, CLAHE
)

import sklearn
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import telebot
import time
import tensorflow as tf
from tensorflow import keras

import lime
from lime import lime_image

import pandas as pd
import pickle

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
  try:
    print("gpus existem")
    print(gpus)
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)
    
    
TELEBOT_TOKEN = "2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA"


img_size = 300
bot = telebot.TeleBot("2058519653:AAG5Kf0Othtye8e13F5WPnBQQSdoCt47ifA")
bot.config['api_key'] = TELEBOT_TOKEN
bot.get_me()

gpus existem
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


{'ok': True,
 'result': {'id': 2058519653,
  'is_bot': True,
  'first_name': 'cnn_covid',
  'username': 'cnn_covid_bot',
  'can_join_groups': True,
  'can_read_all_group_messages': False,
  'supports_inline_queries': False}}

## Augmentation Class

In [2]:
class AugmentationSequence(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, augmentations):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.augment = augmentations

  def __len__(self):
    return int(np.ceil(len(self.x) / float(self.batch_size)))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
    
    aug_x = np.zeros(batch_x.shape)
    for idx in range(batch_x.shape[0]):
      aug = self.augment(image = batch_x[idx,:,:])
      aug_x[idx,:,:] = aug["image"]

    return np.stack((aug_x,) * 3, axis = -1), batch_y


## Load Datasets

In [3]:
file_pi = open('input/train_dataset.pkl', 'rb') 
train_generator =  pickle.load(file_pi)

X_val = np.load('input/X_val.npy')
Y_val = np.load('input/Y_val.npy')

steps_per_epoch = 321

## Create Model

In [4]:
def create_model():
  
  inputs = keras.Input(shape = (img_size, img_size, 3))
  
  base_model = keras.applications.VGG16(
    weights = "imagenet",
    include_top = False,
    input_shape = (img_size, img_size, 3)
  )
  base_model.trainable = False
  
  x = base_model(inputs, training = False)
  x = keras.layers.GlobalAveragePooling2D()(x)
  x = keras.layers.Dense(1024, activation = "relu")(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(1024, activation = "relu")(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(512, activation = "relu")(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Dropout(0.5)(x)
  output = keras.layers.Dense(3, activation = 'softmax')(x)

  model = keras.Model(inputs = inputs, outputs = output)

  return model, base_model

## Fit Model

In [5]:
np.random.seed(587)

reduce_learning_rate = keras.callbacks.ReduceLROnPlateau(
  monitor = "loss", 
  factor = 0.5, 
  patience = 3, 
  verbose = 1
)

model_name = "cache/tl_vgg16_cd.h5"

checkpointer = keras.callbacks.ModelCheckpoint(
  model_name,
  monitor = "val_accuracy",
  verbose = 1, 
  save_best_only = True
)

strategy = tf.distribute.MirroredStrategy()

if (os.path.exists(model_name)):
  model = keras.models.load_model(model_name)
  print("existe")
  
else:
  model, base_model = create_model()
  adam_opt = keras.optimizers.Adam(learning_rate = 0.001 )
  model.compile(optimizer = adam_opt, loss = "categorical_crossentropy", metrics = ["accuracy"])
  start = time.time()    
  fit = model.fit(train_generator, 
    steps_per_epoch = steps_per_epoch, 
    epochs = 50,
    validation_data = (X_val, Y_val),
    callbacks = [
      checkpointer,
      reduce_learning_rate
    ]
  )
  end = time.time()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Epoch 1/50
321/321 [==============================] - 124s 350ms/step - loss: 1.0897 - accuracy: 0.6374 - val_loss: 0.8217 - val_accuracy: 0.6508

Epoch 00001: val_accuracy improved from -inf to 0.65082, saving model to cache/tl_vgg16_cd.h5
Epoch 2/50
321/321 [==============================] - 102s 318ms/step - loss: 0.7653 - accuracy: 0.6976 - val_loss: 0.5623 - val_accuracy: 0.7888

Epoch 00002: val_accuracy improved from 0.65082 to 0.78878, saving model to cache/tl_vgg16_cd.h5
Epoch 3/50
321/321 [==============================] - 100s 312ms/step - loss: 0.6717 - accuracy: 0.7346 - val_loss: 0.4909 - val_accuracy: 0.8176

Epoch 00003: val_accuracy improved from 0.78878 to 0.81761, saving model to cache/tl_vgg16_cd.h5
Epoch 4/50
321/321 [==============================] - 101s 314ms/step - loss: 0.6141 - accuracy: 0.7551 - val_loss: 0.4937 - val_accuracy: 0.8020

Epoch 00004: val_accur


Epoch 00038: val_accuracy improved from 0.86516 to 0.87373, saving model to cache/tl_vgg16_cd.h5
Epoch 39/50
321/321 [==============================] - 103s 319ms/step - loss: 0.4362 - accuracy: 0.8300 - val_loss: 0.3437 - val_accuracy: 0.8683

Epoch 00039: val_accuracy did not improve from 0.87373
Epoch 40/50
321/321 [==============================] - 100s 313ms/step - loss: 0.4274 - accuracy: 0.8349 - val_loss: 0.3475 - val_accuracy: 0.8714

Epoch 00040: val_accuracy did not improve from 0.87373
Epoch 41/50
321/321 [==============================] - 101s 316ms/step - loss: 0.4282 - accuracy: 0.8331 - val_loss: 0.3606 - val_accuracy: 0.8644

Epoch 00041: val_accuracy did not improve from 0.87373

Epoch 00041: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 42/50
321/321 [==============================] - 100s 311ms/step - loss: 0.4214 - accuracy: 0.8347 - val_loss: 0.3394 - val_accuracy: 0.8753

Epoch 00042: val_accuracy improved from 0.87373 to 0.87529, savi

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 9, 9, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              104960

In [7]:
final_train1 = end-start

## Create Finetuning Model

In [8]:
np.random.seed(587)

model_name = "cache/tl_vgg16_finetune_cd.h5"

if (os.path.exists(model_name)):
  print("existe")
  model = keras.models.load_model(model_name)
  
else:
  base_model.trainable = True
  adam_opt = keras.optimizers.Adam(learning_rate = 0.0001)
  model.compile(optimizer = adam_opt, loss = "categorical_crossentropy", metrics = ["accuracy"])
  model.save('cache/tl_vgg16_finetune_cd.h5')
  

### Save times

In [10]:
text_file = open("input/time_train.txt", "wt")
n = text_file.write(str(final_train1))
text_file.close()